Burst Balloons
======

* [Problem Description](#problem_description)
* [Test Cases](#test_cases)
* [Approach 1: Trying all the possibilities](#approach_1)
* [Solution 3: Analytical solution with Least Squares Error minimization](#least-squares-error-minimization)
* [Comparing all 3 solutions](#comparing_solutions)

In this problem we will find the best line that fits a model given by a real dataset. For theory details, please see [Linear Regression Theory](Linear_Regression_Theory.ipynb).

### Problem Description  <a id='problem_description'></a>

Given **n** balloons, indexed from **0** to **n-1**. Each balloon is painted with a number on it represented by array **nums**. You are asked to burst all the balloons. If the you burst balloon **i** you will get **nums[left] $*$ nums[i] $*$ nums[right]** coins. Here **left** and **right** are adjacent indices of **i**. After the burst, the **left** and **right** then becomes adjacent.

Find the maximum coins you can collect by bursting the balloons wisely.

Note:<br>
(1) You may imagine **nums[-1] = nums[n] = 1**. They are not real therefore you can not burst them.<br>
(2) 0 ≤ **n** ≤ 500, 0 ≤ **nums[i]** ≤ 100

Example:

Given: **[3, 1, 5, 8]**<br>
Return: **167**

Source: https://leetcode.com/problems/burst-balloons/description/

### Test Cases <a id='test_cases'></a>

I generated a few test cases so you can test your own implementations:

![](figures/burst_balloon_test_cases.png)


### Approach 1: Trying all the possibilities <a id='approach_1'></a>

One way of solving this problem is to try all possible ways of bursting the balloons and see which order maximizes the final score. This can also be viewed as a **top-bottom** approach.<br>
For the example presented by the problem (balloons [3, 1, 5, 8]), I created the following tree showing all possible cases:

![](figures/burst_balloon_ex1.png)


First we start with the balloons 3, 1, 5 and 8. <br>
We have 4 possible options of bursting the balloons. We could burst balloon #3 or balloon #1 or balloon #5 or balloon #8 as described in the first level from top down in the image.<br>
If we burst balloon 3 we will score 3 points, once $1*3*1=3$ (nums[left] $*$ nums[i] $*$ nums[right]). And as it is the first balloon we are bursting, the accumulated score is 3, shown as **acc** in the chart.<br>

**Observation:** According to Note (2), the first $1$ is used because as the balloon we are bursting is the most left in this sequence, there is no other balloon on its left, therefore we will always consider its left balloon as $1$. This same rule applies for bursting the most right balloon.

Then we are left with balloons 1, 5 and 8 as can be seen in the sequence. So we can continue exploring all the possible options of eliminating the balloons and in the end we observe all achievable final scores which are highlighted in the final branches of the tree.<br>
See that the highest possible score is **167** and can be obtained shooting the balloons 1, 5, 3 and 8 in sequence.

In this approach we will end up with $n!$ scores to decide which one is the highest with complexity equals to $O(n!)$.<br>
Where $n$ is the number of balloons we begin with. In this example we have $n=4$ resulting in $4!=24$ possible scores.<br>
Using this approach our algorithm has $O(24)$ operations.<p>

The code below shows my implementation of this approach.

In [37]:
input = [5,2,3,4,1,9,10] 
# You could also enter the balloons as: 
# input = '3158'

def operate(input, score, max_score):
    # Loop through each  balloon
    for i in range(len(input)):
        newinput = input[:i] + input[i+1:]
        # If you shoot most left balloon, the left balloon's score is 1 by default
        if i == 0: i_= 1 
        else: i_ = int(input[i-1])
        # If you shoot most right balloon, the right balloon's score is 1 by default
        if i == len(input)-1:  ii = 1
        else: ii = int(input[i+1])
        # Calculate score
        new_score = i_*int(input[i])*ii
        # Accumulated score
        acc = new_score + score
        # Uncomment this line to print all possible final scores
        # if (len(newinput) == 0): print('Final score:', acc)
        # If there is no more balloons to shoot, check if achieved score is the highest
        if ((len(newinput) == 0) & (acc > max_score)):
            max_score = acc
        else:
            # Go for next round
            max_score = operate(newinput, acc, max_score)
    return max_score

# Start bursting the balloons
score = operate(input, 0, 0)
# Showing the results
print('Max score:', score)    

Max score: 819


### Approach 2: Using memorization<a id='approach_2'></a>

Another way to solve is to avoid calculating the same scores more than once.<br>
After popping each balloon a new branch of possibilities is produced. The image below highlights the new possible bursting options after each bursting

![](figures/burst_balloon_ex2.png)

For the first bursting we have four possibilities (bursting balloon 3, 1, 5 or 8), leaving us with 4 possible balloons left:  [1,5,8], [3,5,8], [3,1,8], [3,1,5] (shown in red).
For the second bursting we have 12 possibilities: [5,8], [1,8], [1,5], [5,8], [3,8] ... [3,1] (shown in green).
For the 3rd bursting we have 24 possibilities: [8], [5], [8], [1], [5] ... [3] (shown in blue).
For the 4th and last bursting we have only one balloon left, leading us to 24 possibilities: [8], [5], [8], [1], [5] ... [3] (shown in yellow).

If we pay attention we can see that for the second bursting on, some of the options are repeated. The tree below shows that among all possible two-balloon options, we are left with balloons [5,8] twice.

![](figures/burst_balloon_ex3.png)

It is easily seen that other options of 2 balloons also repeats twice: [1,8], [1,5], [3,8] and [3,5]. It means that the operations we are performing in each of those branches are redundant! <br>
If we store those accumulated scores into memory we avoid unnecessary operations and improve our algorithm performance. It means we just have to calculate those scores once.

Each branch has its own accumulated scores. For example, if we consider the two branches with balloons [5,8], we see that it contributes with two gains each. If we burst balloon 5, the gain is 48 and if we burst balloon 8, the gain is 45 as shown below: 

![](figures/burst_balloon_ex4.png)

The gains are the same, because they are generated by popping up the same balloons.<br>

**As our goal is to obtain the highest possible score, we will only store the highest gain**, improving even more the efficiency of our algorithm.<br>
If we follow this appproach, our tree will look like this:






First we start with the balloons 3, 1, 5 and 8. <br>
We have 4 possible options of bursting the balloons. We could burst balloon #3 or balloon #1 or balloon #5 or balloon #8 as described in the first level from top down in the image.<br>
If we burst balloon 3 we will score 3 points, once $1*3*1=3$ (nums[left] $*$ nums[i] $*$ nums[right]). And as it is the first balloon we are bursting, the accumulated score is 3, shown as **acc** in the chart.<br>

**Observation:** According to Note (2), the first $1$ is used because as the balloon we are bursting is the most left in this sequence, there is no other balloon on its left, therefore we will always consider its left balloon as $1$. This same rule applies for bursting the most right balloon.

Then we are left with balloons 1, 5 and 8 as can be seen in the sequence. So we can continue exploring all the possible options of eliminating the balloons and in the end we observe all achievable final scores which are highlighted in the final branches of the tree.<br>
See that the highest possible score is **167** and can be obtained shooting the balloons 1, 5, 3 and 8 in sequence.

In this approach we will end up with $n!$ scores to decide which one is the highest with complexity equals to $O(n!)$.<br>
Where $n$ is the number of balloons we begin with. In this example we have $n=4$ resulting in $4!=24$ possible scores.<br>
Using this approach our algorithm has $O(24)$ operations.<p>

The code below shows my implementation of this approach.

In [59]:
class Solution(object):
    def maxCoins(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        nums = [1] + nums + [1]
        n = len(nums)
        dp = [[0] * n for _ in range(n)]
        def calculate(i, j):
            if dp[i][j] or j == i + 1: # in memory or gap < 2
                return dp[i][j]
            coins = 0
            for k in range(i+1, j): # find the last balloon
                coins = max(coins, nums[i] * nums[k] * nums[j] + calculate(i, k) + calculate(k, j))
            dp[i][j] = coins
            return coins

        return calculate(0, n-1)

sol = Solution()

import random

for j in range(4):
    n = 9
    array = []
    for i in range(n):
        array.append(random.randint(1, 10))

    res  = sol.maxCoins(nums=array)
    print('Input: %s Output: %s' % (array, res))


# sol.maxCoins(nums=[5,2,3,4,1,9,10] )

Input: [10, 4, 10, 9, 7, 9, 8, 1, 1] Output: 3507
Input: [1, 6, 2, 1, 9, 5, 4, 1, 9] Output: 1305
Input: [1, 10, 3, 1, 8, 10, 8, 7, 5] Output: 2844
Input: [6, 9, 8, 3, 5, 2, 1, 10, 1] Output: 1980
